In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os, sys
import pandas as pd
import numpy as np
import random
import math
from future.utils import listvalues
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statistics
import pickle
import hssm
warnings.filterwarnings("ignore")

/Users/zhuojunyu/miniconda3/envs/hssm_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def fitHDDM(datatable):
    
    nw_data = pd.DataFrame()
    nw_data['response'] = datatable['decision']
    nw_data['response'] = nw_data['response'].map({'left':1,'right':-1})
    nw_data['rt'] = datatable['decisiondurationplusdelay']/1000.0

    model = hssm.HSSM(data=nw_data,model='ddm')
    hddm_model = model.sample()

    return hddm_model

In [6]:
nseeds = len(next(os.walk('../Data/'))[1])
hddm_results = pd.DataFrame()
data_dir_all = "../Data/"

for i in np.arange(1,nseeds+1):
    seed_num = i
    data_dir = data_dir_all+"network_"+str(seed_num)+"/"
    nw_data = pickle.load(open(data_dir+"network_"+str(seed_num)+".pickle","rb"))
    nw = nw_data[0]
    datatable = nw['datatables']
    
    hddm_model = fitHDDM(datatable)
    
    hddm_z = hddm_model.posterior['z'].values
    hddm_a = hddm_model.posterior['a'].values
    hddm_t = hddm_model.posterior['t'].values
    hddm_v = hddm_model.posterior['v'].values

    di = dict()
    di['seed'] = seed_num
    di['mean_a'] = hddm_a.mean()
    di['std_a'] = hddm_a.std()
    di['mean_v'] = hddm_v.mean()
    di['std_v'] = hddm_v.std()
    di['mean_t'] = hddm_t.mean()
    di['std_t'] = hddm_t.std()
    di['mean_z'] = hddm_z.mean()
    di['std_z'] = hddm_z.std()
    di['rt'] = statistics.median(datatable.decisionduration) 
    di = pd.DataFrame([di])
    print(di)
    
    hddm_results = pd.concat([hddm_results,di])

data_dir_store = r"../DDM/" 
hddm_results.to_csv(data_dir_store+"hddm_results.csv")

Model initialized successfully.
Using default initvals. 



Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [z, a, t, v]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 3 seconds.
100%|██████████| 4000/4000 [00:00<00:00, 30451.82it/s]

   seed  mean_a     std_a   mean_v    std_v    mean_t    std_t    mean_z  \
0     1  0.1557  0.011034  1.87917  1.01166  0.328088  0.00094  0.464785   

      std_z  rt  
0  0.047941  98  
